In [1]:
def load_unigrams(file):
    K = 5
    unigrams = {}
    bigrams = {}
    with open(file, 'r') as f:
        for line in f:
            count = int(line.split()[0])
            w1 = line.split()[1]
            w2 = line.split()[2]
            if count >= K:
                
                if w1 not in bigrams:
                    bigrams[w1] = {}
                bigrams[w1][w2] = count
                
                if w1 not in unigrams:
                    unigrams[w1] = 1
                if w2 not in unigrams:
                    unigrams[w2] = 1
                unigrams[w1] += 1
                unigrams[w2] += 1
    return unigrams,bigrams

In [2]:
unigrams,bigrams = load_unigrams("poleval_2grams.txt")

In [3]:
N = 0
for u in unigrams:
    N += unigrams[u]
print(N)

14058114


# Tagi i Ngramy tagow

In [4]:
def load_supertags(file):
    d_suf_words = {}
    d_tags = {}
    d_words = {}
    with open(file, 'r') as f:
        for line in f:
            t = line.split()[1]
            w = line.split()[0]
            ws = ("^" + w)[-3:]
            d_words[w] = t
            if ws not in d_suf_words:
                d_suf_words[ws] = {}
            if t not in d_suf_words:
                d_suf_words[ws][t] = 0
            d_suf_words[ws][t] += 1
            if t not in d_tags:
                d_tags[t] = {}
            if w in unigrams:
                d_tags[t][w] = unigrams[w]
            else:
                d_tags[t][w] = 1
    return (d_suf_words, d_tags, d_words)

In [5]:
d_suf_words, d_tags, d_words = load_supertags("supertags.txt")
def gen_tag(d_t):
    tags = list (d_t.keys())
    prob = list ( d_t.values() )
    sum_prob = sum (prob)
    r = np.random.choice(len (tags) , p = [p_r/sum_prob for p_r in prob])
    return tags[r]

In [6]:
def load_taggrams(file):
    K = 5
    t_unigrams = {}
    t_bigrams = {}
    with open(file, 'r') as f:
        for line in f:
            count = int(line.split()[0])
            w1 = line.split()[1]
            w2 = line.split()[2]
            if w1 in d_words:
                w1 = d_words[w1]
            else:
                count = 0
            if w2 in d_words:
                w2 = d_words[w2]
            else:
                count = 0
            if count >= K:
                
                if w1 not in t_bigrams:
                    t_bigrams[w1] = {}
                t_bigrams[w1][w2] = count
                
                if w1 not in t_unigrams:
                    t_unigrams[w1] = 1
                if w2 not in t_unigrams:
                    t_unigrams[w2] = 1
                t_unigrams[w1] += 1
                t_unigrams[w2] += 1
    return t_unigrams, t_bigrams

In [7]:
t_unigrams,t_bigrams = load_taggrams("poleval_2grams.txt")

In [8]:
T = 0
for t in t_unigrams:
    T += t_unigrams[t]
print(T)

9640981


# KOLOKACJE

In [15]:
from math import log as ln
def PPMI(word):
    ans = []
    for w in bigrams[word]:
        pmi = ln(bigrams[word][w] * N / (unigrams[w] * unigrams[word]) )
        ans.append( (max(0, pmi) , w) )
    ans.sort(reverse = True)
    return ans  
def PSM(word):
    ans = []
    for w in bigrams[word]:
        psm = bigrams[word][w] * (ln(bigrams[word][w]) - ln( (unigrams[w] * unigrams[word]) / N)  - 1)
        ans.append( (psm , w) )
    ans.sort(reverse = True)
    return ans
def PPMI_mod(word):
    ans = []
    for w in bigrams[word]:
        pmi = ln(bigrams[word][w] * pow(N, 0.75) / (pow (unigrams[w], 0.75) * unigrams[word]) )
        ans.append( (max(0, pmi) , w) )
    ans.sort(reverse = True)
    return ans  

def gramatic_ws(word):
    ans_w = []
    ans_t = []
    tag = d_words[word]
    for w in bigrams[word]:
        pmi = ln(bigrams[word][w] * N / (unigrams[w] * unigrams[word]) )
        ans_w.append( (max(0, pmi) , w) )
    #print(t_bigrams[tag])
    for t in t_bigrams[tag]:
        pmi = ln(t_bigrams[tag][t] * T / (t_unigrams[t] * t_unigrams[tag]) )
        ans_t.append( (max(0, pmi) , t) )
    ans_w.sort(reverse = True)
    ans_t.sort(reverse = True)
    ans_t = [t[1] for t in ans_t]
    
    k = 300
    ans = []
    while(len(ans) < 10 and k < 2000):
        ans.clear()
        for a in ans_w:
            if a[1] in d_words:
                #print("yo")
                if d_words[a[1]] in ans_t[:k]:
                    ans.append(a)
        k += 100
    return ans
    

In [16]:
words = ['lampa' , 'piłka', 'ogrzewanie', 'kuchenka', 'samochód', 'naprawiać', 'komputer',
        'telefon', 'zamek', 'golf', 'programowanie']
print("PPMI\n")
for w in words:
    i = 0
    ans = PPMI(w)
    print(w,":")
    for a in ans:
        if i > 9:
            break
        print(a)
        i += 1
    print("\n")

PPMI

lampa :
(14.020054663809493, 'błyskowa')
(13.833136316668767, 'sufitowa')
(13.61700793656561, 'wisząca')
(13.225931565909546, 'biurkowa')
(13.200613757925256, 'wiszaca')
(12.715105942143555, 'rtęciowa')
(12.623538748618065, 'podłogowa')
(12.5327843853496, 'stojaca')
(12.5327843853496, 'sollux')
(12.427423869691774, 'acetylenowa')


piłka :
(14.963018125472647, 'nożna')
(14.30955774989451, 'siatkowa')
(12.738179399284755, 'ręczna')
(12.279766116858495, 'odbiła')
(12.264017759890356, 'zatrzepotała')
(12.081696203096401, 'golfowa')
(11.927545523269142, 'przeleciała')
(11.704401971954933, 'poszybowała')
(11.676231094988237, 'mikasa')
(11.650913287003947, 'uderzona')


ogrzewanie :
(13.44879338740079, 'podłogowe')
(12.655562748283863, 'piecowe')
(12.601495527013586, 'etażowe')
(12.514484150023957, 'nadmuchowe')
(12.147711841634615, 'gazowe')
(12.13718991888249, 'postojowe')
(12.024180161978764, 'kominkowe')
(11.972886867591214, 'nawiewne')
(11.890648769354241, 'olejowe')
(11.502883238

In [17]:
print("PPMI_MOD\n")
for w in words:
    i = 0
    ans = PPMI_mod(w)
    print(w,":")
    for a in ans:
        if i > 9:
            break
        print(a)
        i += 1
    print("\n")

PPMI_MOD

lampa :
(10.426762078696136, 'sufitowa')
(10.39116737795128, 'wisząca')
(10.251950680102782, 'błyskowa')
(9.385907064089889, 'biurkowa')
(9.259222979078558, 'wiszaca')
(9.257794243019879, 'podłogowa')
(8.773715163296858, 'rtęciowa')
(8.692759883529943, 'stojaca')
(8.692759883529943, 'sollux')
(8.67117814929372, 'stołowa')


piłka :
(11.75107002964702, 'nożna')
(10.770526449156337, 'siatkowa')
(9.601507576655107, 'ręczna')
(8.926219152302666, 'odbiła')
(8.495913776183643, 'zatrzepotała')
(8.472632281686273, 'przeleciała')
(8.140305424249703, 'golfowa')
(8.076201935232076, 'poszybowała')
(8.056096098437221, 'uderzona')
(7.90413287265074, 'meczowa')


ogrzewanie :
(10.312121564771143, 'podłogowe')
(9.198894041175947, 'gazowe')
(8.988825041604192, 'piecowe')
(8.911349360268158, 'postojowe')
(8.746380166317246, 'nadmuchowe')
(8.660104748166889, 'etażowe')
(8.645612332783688, 'kominkowe')
(8.512080940159166, 'olejowe')
(8.359615017057886, 'olejem')
(8.260568771713054, 'nawiewne')



In [18]:
print("Gramatycznie-slowowo\n")
for w in words:
    i = 0
    ans = gramatic_ws(w)
    print(w,":")
    for a in ans:
        if i > 9:
            break
        print(a)
        i += 1
    print("\n")

Gramatycznie-slowowo

lampa :
(11.231173111459748, 'sprzedawana')
(10.85880795177793, 'gazowana')
(9.82918853459677, 'tył')
(9.719373668589565, 'inspirowana')
(8.713851802987467, 'sprawdzi')
(8.547736244906565, 'wykonana')
(7.531077721997646, 'przeznaczona')
(5.430285029574952, 'powinna')
(2.710615353991061, 'czy')
(2.351538841559385, 'to')


piłka :
(11.650913287003947, 'uderzona')
(11.388549022536456, 'kopana')
(11.008219887699834, 'wpadła')
(10.679897655440545, 'odbita')
(10.472258290662301, 'pięcioosobowa')
(9.830404404489906, 'wybita')
(9.248482859040184, 'odbije')
(8.906623000886594, 'dotknie')
(7.595872774740876, 'wykonana')
(7.367671612196228, 'przeznaczona')


ogrzewanie :
(11.381930627927815, 'olejem')
(11.26172118152859, 'piecem')
(9.631081061443886, 'gazem')
(8.451581113636676, 'piec')
(8.270762186727255, 'kabiny')
(8.24686786652782, 'domu')
(7.358573423687118, 'budynku')
(7.145009589159593, 'domów')
(7.139253968811312, 'wody')
(5.143764763824602, 'co')


kuchenka :
(16.435

In [19]:
print("PSM\n")
for w in words:
    i = 0
    ans = PSM(w)
    print(w,":")
    for a in ans:
        if i > 9:
            break
        print(a)
        i += 1
    print("\n")

PSM

lampa :
(4352.867738115136, 'wisząca')
(2669.2923538671034, 'sufitowa')
(848.5183286491188, 'podłogowa')
(782.4616844893472, 'tylna')
(768.1832251647601, 'błyskowa')
(564.2234027292728, 'stojąca')
(527.094564819041, 'stołowa')
(432.38091953672125, 'ścienna')
(399.0157513469302, 'sprzedawana')
(360.1045221166013, 'z')


piłka :
(23038.97990702987, 'nożna')
(3087.817397975526, 'siatkowa')
(2828.9012352276254, 'ręczna')
(998.7836478375647, 'po')
(889.9053017879385, 'trafiła')
(845.1362033353969, ',')
(721.9050314789436, 'odbiła')
(489.17018740154964, 'wodna')
(470.3863347218922, 'wpadła')
(463.38382168142755, 'jest')


ogrzewanie :
(4357.077685590277, 'podłogowe')
(2382.96316795858, ',')
(2251.8377920101925, 'gazowe')
(1158.7514640116287, '.')
(987.823189529867, 'elektryczne')
(941.5747503059561, 'centralne')
(936.933865835861, 'i')
(735.0545346462442, 'postojowe')
(732.6555229727189, 'miejskie')
(726.735143954947, 'olejem')


kuchenka :
(2639.4855272279265, 'mikrofalowa')
(1164.7499